In [1]:
import pandas as pd
import re
import numpy as np
import emoji
import contractions
import unidecode
from tqdm import tqdm
pd.options.display.max_colwidth=200
pd.options.display.max_rows=None

In [2]:
data = pd.read_csv('tweets.csv')

/var/folders/2k/fhkt65ls68nbd5xt0lj3sbww0000gn/T/ipykernel_59839/3005909632.py:2: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/rosni/Library/CloudStorage/OneDrive-NJIT/Dissertation/curbside/Data/Python files/Final_data.csv')


371272

In [3]:
data.columns

Index(['index', 'author id', 'created_at', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'Category', 'channel'],
      dtype='object')

In [6]:
data['channel'].value_counts()

delivery    329103
curbside     42162
Name: channel, dtype: int64

In [7]:
data['Category'].value_counts()

Consumer         8379
Advertisement    8092
Name: Category, dtype: int64

In [8]:
data = data.drop_duplicates(subset='tweet')
len(data)

368414

In [9]:
data['Category'].value_counts()

Consumer         8377
Advertisement    8092
Name: Category, dtype: int64

In [10]:
data.head()

,index,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,Category,channel,new_tweet
0,5,47199132.0,2015-01-21 19:23:29+00:00,,5.580000e+17,en,0.0,0.0,0.0,0.0,Twitter for iPhone,"@jhineline meanwhile up the street, grocery shoppers park at trader joe's using the convenienttent curbside.",Consumer,curbside,"@jhineline meanwhile up the street, grocery shoppers park at trader joe's using the convenienttent curbside."
1,8,34079088.0,2015-01-13 15:29:39+00:00,,5.550000e+17,en,1.0,0.0,0.0,2.0,Twitter for iPhone,".@walmart is in trial phase of online orderering + curbside pickup with grocery, truly a disruptor and convenienttent! #nrf15 #futureofretail",Consumer,curbside,".@walmart is in trial phase of online orderering + curbside pickup with grocery, truly a disruptor and convenienttent! #nrf15 #futureofretail"
2,9,119482829.0,2015-01-09 23:01:37+00:00,,5.540000e+17,en,0.0,0.0,1.0,0.0,Twitter for iPhone,thank you @harristeeter for providing a curbside grocery pick up. i love you. that's all.,Consumer,curbside,thank you @harristeeter for providing a curbside grocery pick up. i love you. that's all.
3,10,718558608.0,2015-01-08 04:08:17+00:00,,5.530000e+17,en,0.0,0.0,0.0,0.0,Twitter for iPhone,i can't get to the grocery store weekly to buy food &amp; you want me to haul trash across town? support curbside #recycle,Advertisement,curbside,i can't get to the grocery store weekly to buy food &amp; you want me to haul trash across town? support curbside #recycle
4,14,16347551.0,2015-02-24 19:47:44+00:00,01fbe706f872cb32,5.700000e+17,en,0.0,0.0,2.0,0.0,Twitter for iPhone,loving @peapod delivers i was in &amp; out of the grocery store with a week of food using their curbside pick up in 2 minutes. #gamechanging,Consumer,curbside,loving @peapod delivers i was in &amp; out of the grocery store with a week of food using their curbside pick up in 2 minutes. #gamechanging


In [12]:
data['created_at'] = pd.to_datetime(data['created_at'])
data['Year'] = data['created_at'].dt.year
data['Month'] = data['created_at'].dt.month_name()
data['Day'] = data['created_at'].dt.day_name()

In [13]:
print(min(data['created_at']))
print(max(data['created_at']))

2015-01-01 00:07:25+00:00
2022-11-29 23:51:09+00:00


In [14]:
count = data['tweet'].str.split().str.len()
print("Total number of words: ", count.sum())
print("Average number of words per tweet: ", round(count.mean(),2))
print("Minimum number of words per tweet: ", round(count.min(),2))
print("Maximum number of words per tweet: ", round(count.max(),2))
print("Maximum number of words per tweet: ", round(count.max(),2))
print("Median number of words per tweet: ", round(count.median(),2))

Total number of words:  9761584
Average number of words per tweet:  26.5
Minimum number of words per tweet:  2
Maximum number of words per tweet:  108
Maximum number of words per tweet:  108
Median number of words per tweet:  23.0


In [15]:
df = data[data['created_at'] >= '2022-01-01']
len(df)

30047

In [16]:
data = data.reset_index()

In [17]:
data.columns

Index(['level_0', 'index', 'author id', 'created_at', 'geo', 'id', 'lang',
       'like_count', 'quote_count', 'reply_count', 'retweet_count', 'source',
       'tweet', 'Category', 'channel', 'new_tweet', 'Year', 'Month', 'Day'],
      dtype='object')

In [18]:
data = data.drop('level_0', axis=1)
data = data.drop('index', axis=1)

In [19]:
data.columns

Index(['author id', 'created_at', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'Category', 'channel', 'new_tweet', 'Year', 'Month', 'Day'],
      dtype='object')

### Convert to lower case

In [20]:
def lower_case(txt):
    return txt.lower()
data['tweet'] = data['tweet'].apply(lambda x : lower_case(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

@cdcarter13 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
@padme_kenobi @kaylive @fakemorganhope @itsaronbitch @robwhite010 @davidfholt they also say it’s probably safer to get food from a restaurant curb side versus grocery shopping right now. less exposure. they shouldn’t close. also workers should watch hands after each curbside delivery.


### Get number of tweets with retweet

In [21]:
#get number of tweets with retweet
def retweet_count(text):
    if 'cc@' in text:
        return True
    else:
        return False

df = data['tweet'].apply(lambda x: retweet_count(x))
df.value_counts()

False    368412
True          1
Name: tweet, dtype: int64

### Get all html tags in tweet

In [22]:
def get_html_tags(text):
    tags = []
    html = re.findall('<.*?>',text)
    if html != None:
        tags.extend(html)
    return tags

html_tags = data['tweet'].apply(lambda x: get_html_tags(x))
html_tags = [i for i in html_tags if len(i)!=0]
html_tags

[]

### Get all markdown links in tweet

In [23]:
def get_html_tags(text):
    tags = []
    html = re.findall(r'\[.*?\]\(.*?\)',text)
    if html != None:
        tags.extend(html)
    return tags

html_tags = data['tweet'].apply(lambda x: get_html_tags(x))
html_tags = [i for i in html_tags if len(i)!=0]
html_tags

[['[start using drones](https://t.co/zs7mt3cr8o)']]

### Get html elements in the tweets

In [24]:
html_elements = []
for i in range(len(data)):
    regexp = "&.+?;" 
    html_elements.extend(re.findall(regexp,data['tweet'][i]))
html_elements = set(html_elements)

from html import unescape
for i in html_elements:
    print(i,"=", unescape(i))
    break

&lt; = <


### Replace html elements in the tweets

In [25]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&amp;', '&', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&gt;', '>', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub('&lt;', '<', x))

### Remove mentions

In [26]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('@(\w+)', '', x))
print(data['tweet'][5003])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.


### Replace URL with 'hyperlink'

In [27]:
print(data['tweet'][50015])

 amazon has grocery delivery??? where the hell have i been???


In [28]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r'http\S+', 'hyperlink', x))
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r'www\.[^ ]+', 'hyperlink', x))
print(data['tweet'][5003])
print(data['tweet'][50015])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
 amazon has grocery delivery??? where the hell have i been???


In [29]:
def count_link(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'hyperlink':
            count += 1
    return count

def count_i(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'i':
            count += 1
    return count

def count_my(data):
    words = ' '.join(data['tweet']).split()
    count = 0
    for word in words:
        if word == 'my':
            count += 1
    return count



In [30]:
print(count_link(data), count_i(data), count_my(data))

194361 167064 87123


### Replace email ids with keyword email

In [31]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', r'email',x))
print(data['tweet'][5003])
print(data['tweet'][5006])

 my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.
      they also say it’s probably safer to get food from a restaurant curb side versus grocery shopping right now. less exposure. they shouldn’t close. also workers should watch hands after each curbside delivery.


In [32]:
print(count_link(data), count_i(data), count_my(data))

194361 167064 87123


### Check number of tweet with emojis

In [33]:
def check_emoji(text):
    emo = emoji.UNICODE_EMOJI
    emojis = {**emo['en'],**emo['es'],**emo['pt'],**emo['it'],**emo['fr'],**emo['de']}
    allchars = [i for i in text]
    emoji_list = [c for c in allchars if c in emojis]
    words = text.split()
    contain_emoji = False
    for word in words:
        if word in emoji_list:
            contain_emoji = True
        else:
            contain_emoji = False
    return contain_emoji

tqdm.pandas()
count_emoji = data['tweet'].progress_apply(lambda x: check_emoji(x))
print(count_emoji.value_counts().to_dict())
#print(f'The {count_emoji.value_counts()[1]/count_emoji.value_counts().sum()} of the tweets have emojis')

100%|█████████████████████████████████| 368413/368413 [02:18<00:00, 2664.10it/s]

{False: 357119, True: 11294}


In [34]:
emoji_dict = {'😩': 'tired', '🤦': 'embarassment', '😅': 'happy', '🤣': 'funny', '😎': 'awesome','😂': 'happy', '🙌': 'joy', '👀': 'curious', '👌': 'approval', '🤢': 'disgust', '🙂': 'happy', '😭': 'unhappy', '💵': 'dollar',  '❤': 'love', '🤷': 'ignorance','😬': 'embarassment', '✔': 'approval', 
              '😃': 'happy', '🎯': 'success', '😍': 'love','😤': 'frustration', '🙏': 'gratitude','👏': 'applaud', '👍': 'approval', '🤔': 'thinking', '🙃': 'silliness', '💕': 'love', '🎉': 'celebration','🤩': 'fascinate', '😖': 'frustration', '🤬': 'angry', '🤗': 'happy', '💃': 'happy', '🤯': 'frustration', '😑': 'frustration','✨': 'excitement', '💞': 'love', '🙄': 'disapproval', '😉': 'joke', '😄': 'happy', '💜': 'love', '😡': 'angry','😁': 'happy', '😱': 'fearful:', '😒': 'unhappy', 
              '😋': 'tasty', '😐': 'frustration', '✌': 'peace', '😆': 'funny','😇': 'blessing', '😔': 'regretful', '💯': 'agree','🙈': 'disbelieving', '😕': 'confused', '😊': 'happy','😫': ':tired', '💰': 'money','🤪': 'funny', '😿': 'sad', '😣': 'frustration', '🥴': 'confused', '🤞': 'luck','🙅': 'disapproval', '🤘': 'love', '💩': 'angry','😳': 'embarrassment',  '💛': 'love', '☹': 'sad','😜': 'funny','🥰': 'love', '🙆': 'approval', '🤤': 'longing', '😏': 'smug', 
              '💚': 'love', '🥺': 'pleading', '🤥': 'lying','💖': 'love',  '🧐': 'ponder', '😥': ':sad', '🙇': 'apology', '😢': 'sad','🤕': 'hurt', '☺': 'happy', '😶': 'silence', '😧': 'sad', '💔': 'sad','♥': 'love','🤓': 'nerd', '😲': 'surprise', '🧡': 'love', '🚫': ':prohibited:','😰': ':anxious', '😝': 'funny', '👎': 'disapproval','😀': 'happy', '🌞': 'happy','😵': 'disbelief','💙': 'love', '😯': 'surprise', '🙁': 'angry','😞': ':disappoint', '😘': 'love', '😺': 'happy',
              '🤨': 'confused', '😠': 'angry', '😪': ':sleepy','😦': 'sad', '💗': 'love','😓': 'frustration', '😨': ':fearful', '😌': ':relieved', '❣': 'love',  '🤡': 'funny','😮': 'amazed', '🥳': 'happy','😈': 'trouble',  '🤮': 'disgust', '😴': ':sleeping','😻': 'love', '🥶': 'cold', '🤭': 'embarrassment,', '🖕': 'angry', '💘': 'love', '🤑': 'money','🥲': 'gratitude', '🤟': 'love', '💫': 'dizzy', '😟': 'worry', '🖤': 'love','💸': 'money', '😛': 'fun','🙍': 'disapproval', 
              '💟': 'love', '🙀': 'tired', '🤒': 'ill', '😙': 'love', '💝': 'love', '👿': 'angry', '👪': 'family', '🤰': 'pregnant woman',  
              '😸': 'happy', '🤐': 'speechless', '🤫': 'quiet','🥱': 'tired', '💢': 'anger'}

In [35]:
import json

In [36]:
#read dictionary from json file
with open("../../Supplementary data/emoji_dictionary.json","r") as f:
    emoji_dict = json.load(f)
len(emoji_dict)

145

In [37]:
print(data['tweet'][8])
print(data['tweet'][16])

walmart testing curbside grocery pickup in huntsville hyperlink
proudly introducing "shopping for survivors" - a free grocery shopping and curbside pick-up service for cancer patients. thanks !


In [38]:
#Remove emojis
tqdm.pandas()
def give_emoji_free_text(text, emoji_dict):
    emo = emoji.UNICODE_EMOJI
    emojis = {**emo['en'],**emo['es'],**emo['pt'],**emo['it'],**emo['fr'],**emo['de']}
    all_lst = []
    words = text.split()
    for word in words:
        lst = []
        allchars = [i for i in word]
        for j in allchars:
            if j in emojis:
                if j in emoji_dict:
                    lst.append(' ')
                    lst.append(emoji_dict[j])
            else:
                lst.append(j)
        all_lst.append(''.join(lst))
    return ' '.join(all_lst)

data['tweet'] = data['tweet'].progress_apply(lambda x: give_emoji_free_text(x,emoji_dict))
print(data['tweet'][8])
print(data['tweet'][16])

100%|█████████████████████████████████| 368413/368413 [02:25<00:00, 2528.54it/s]

walmart testing curbside grocery pickup in huntsville hyperlink
proudly introducing "shopping for survivors" - a free grocery shopping and curbside pick-up service for cancer patients. thanks !


In [39]:
print(count_link(data), count_i(data), count_my(data))

194525 167083 87128


### Remove contractions

In [40]:
import contractions
data['tweet'] = data['tweet'].apply(lambda x: contractions.fix(x))
print(data['tweet'][5003])

my wife stopped at the grocery store for curbside pickup where they put your orderer in your trunk with no contact. while she was there she saw tons of boomers going into the store with no protection at all.


In [41]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove stopwords

In [42]:
#remove stopwords
words = ' '.join(data['tweet']).split()
print(len(words))
from nltk.corpus import stopwords
stop = ['what','which','this','that',"that'll",'these','those','is','are','was','were','be','been','being','have',
              'has','had','having','do','does','did','doing','a','an','the', 'and','but','if','or','because','as','until',
              'while','of','at','by','for','with','about','against','between','into','through','during','before','after',
              'above','below','to','from','up','down','in','out','on','off','over','under','again','further','then',
              'once','here','there','when','where','why','how','all','any','both','each','few','more','most','other',
              'some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just',
              'don',"don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',
              "couldn't", 'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",
              'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',
              "wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't"]
data['tweet'] =  data['tweet'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
print(data['tweet'][5003])
print(data['tweet'][16])
words = ' '.join(data['tweet']).split()
print(len(words))

9705812
my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
proudly introducing "shopping survivors" - free grocery shopping curbside pick-up service cancer patients. thanks !
6708406


In [43]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove next line characters

In [44]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('\n', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
they also say it probably safer get food restaurant curb side versus grocery shopping right now. less exposure. they close. also workers watch hands curbside delivery.


In [45]:
print(count_link(data), count_i(data), count_my(data))

194525 213515 87128


### Remove accented characters to their ASCII values

In [46]:
def accented_to_ascii(text):
    text = unidecode.unidecode(text)
    return text
data['tweet'] = data['tweet'].apply(lambda x: accented_to_ascii(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact. she she saw tons boomers going store protection all.
they also say it probably safer get food restaurant curb side versus grocery shopping right now. less exposure. they close. also workers watch hands curbside delivery.


In [47]:
print(count_link(data), count_i(data), count_my(data))

194578 213532 87129


### Remove special characters

In [48]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub('[!"#%“[•►–&”\'()*+,-/:;<=>?@\\^_{|}~`]', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][16])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact  she she saw tons boomers going store protection all 
proudly introducing  shopping survivors    free grocery shopping curbside pick up service cancer patients  thanks  


In [49]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Remove spaces

In [50]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(' +', ' ', x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all 
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery 


In [51]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Replacing currency symbol by word

In [52]:
data['tweet'] = data['tweet'].apply(lambda x : re.sub(r"\$(\d+)", r"\1 dollars", x))
print(data['tweet'][780])

 ucwradio walmart expands its curbside grocery pickup service you s hyperlink technology


In [53]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Remove numbers

In [54]:
import num2words
def remove_num(txt):
    return ' '.join(' ' if i.isdigit() else i for i in txt.split())

data['tweet'] = data['tweet'].apply(lambda x : remove_num(x))  
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [55]:
print(count_link(data), count_i(data), count_my(data))

195617 218051 87708


### Find words with multiple same consecutive letters

In [56]:
two_consecutive_letters = []
three_consecutive_letters = []
count_2 = 0
count_3 = 0
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if re.search(r'((\w)\2{2,})', word):
            count_3 += 1
            if word not in three_consecutive_letters:
                three_consecutive_letters.append(word)
        elif re.search(r'((\w)\2{1,})', word):
            count_2 += 1
            if word not in two_consecutive_letters:
                two_consecutive_letters.append(word)
print(count_2, count_3)

100%|████████████████████████████████| 368413/368413 [00:17<00:00, 21271.17it/s]

809575 3794


In [57]:
words_to_remove = to_remove = ['bbb', 'litttt', 'daaaaa', 'holllla', 'ggg', 'cccsc', 'oooo', 'ppp', '1000xs', 'ooooo', 'ffffff', 'haaa','brrr', 'econ1110', 'a444', 'mmmpanadas', 'toowaahhhh', 'mmmmmmmm', 'mmmagtweets', 'zzzzzz', 'mmmmm', 'ifttt', 'from1000km', 'bellletstalk', 'xxx', 'mmm', 'zzzz', 'vrai777', 'tracycopygrrrl', 
             'zzzzzzzzzzzzzzz', 'ooo', 'jefffrey', 'whole30fff', 'yyccc', 'mmmm', 'mmmmmm', 'hrmmm', 'fffuuuu', '1000mg', 'aaa', 'www', '7771x29', 'ffffffffffff', '2000s', 'taaac', 'brandongaillle', 'niggaaaaa','$qqq', 'heee', 'ummmmm', 'paullly', 'theyllgetcontactlessintheyear3000', 'hbuuu', 'sumkt444', 'nascarvvvrroooommmmmmm', 'kkkempsey', 'ttttttt', 'tttt', 'oooooo', 'lorderdddd', 'teamfccc', 
             'wwwwwwwwwww', '1000s', 'lll', 'ieeeorg', 'reeee', 'ltdjnrtnnnbggngvcfi', 'wwww', 'hhh', 'onnnnnnn','hheee', 'mmmmmmm', 'xxxx', '1000000000x', 'eee', 'ffbbbllaarrrrrrnnttts', 'wlll', 'pcccommunitymarkets',
             'paccc', 'supportpaccc', 'amoooo', 'cj333', 'balllin', '000mph', 'reeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee','squeee', '000s', 'hhhh', 'rdockmatthews111', '5000sq', '1000x', 'aaaaaaa', 'booo19', 'ggggggggg','mysicklecelllife', 'faaaaa', 'maccleennnny', 'hmmmmm', 'aaaaaaaaaaaa', '000ppl', 'hmmmyess', 'aaaaaa','bestnyaaaa', 'aaaa', '000sq', '2000k', 'ccc', 'eeee', 'ooota', 'neee', 'waangooonline', 'hal2000bot',
             'vcccares', '$glittergrl2000', 'nuuu', 'jaaahaaack', 'breeellery', 'aaaaaaaaaaa', 'bbbslc', 'hhhhhhhhh','3000th', 'sss', 'pleading$sbreezyyy', 'siaaa', 'aaaaaaaa', '15902201009000473m', '15904058179022263m', 'zzzzzzz', 'zzzzzzzzzzzzzzzzz', '000th']

In [58]:
total_count = 0
for i in tqdm(range(len(data))):
    count = 0
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in words_to_remove:
            count += 1
    total_count += count
total_count            

100%|████████████████████████████████| 368413/368413 [00:08<00:00, 45062.32it/s]


415

In [59]:
def remove_words(text, words_to_remove):
    words = text.split(' ')
    new_words = []
    for word in words:
        if word not in words_to_remove:
            new_words.append(word)
    return ' '.join(new_words)
data['tweet'] = data['tweet'].progress_apply(lambda x : remove_words(x, words_to_remove))  
print(data['tweet'][5003])
print(data['tweet'][5006])                   

100%|████████████████████████████████| 368413/368413 [00:07<00:00, 49712.97it/s]

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [60]:
replace_list = {'hummmm': 'hum','alll':'all','allll':'all','alllll':'all','buttt':'but', 'butttt':'but','shoppinggg':'shopping','lifeee': 'life', 'ehhhhhhhh': 'eh','hourrr': 'hour', 'weeeeeeeeeeeeeeeeeeeeeeeeee': 'we', 'buuuuuuuuut': 'but','ohhh': 'oh', 'sooo': 'so', 'superbowlllll': 'superbowl', 'damnnnn': 'damn', 'verrrry': 'very', 'oooh': 'oh', 'aahhhh': 'ah', 'fuckkkk': 'fuck','yooo': 'yo', 'effffffffffff': 'eff', 
                'okurrrrrrrr': 'ok', 'bizzzznesss': 'business', 'waaaaayyy': 'way', 'soooo': 'so','fffffffuuuuuuck': 'fuck', 'hiiii': 'hi', 'ssoooo': 'so', 'lmaooo': 'lmao', 'mannnnn': 'man', 'businesssupportingbusiness': 'business supporting business', 'eeeek': 'ek', 'alll': 'all', 'pssst': 'past', 'cccp': 'cp', 'annnnd': 'and','ummm': 'umm','hmmm': 'hmm', 'wonderfullly': 'wonderfully', 'smallbusinessstrong': 'small business strong','mayyyyyybe': 'may be', 'waaaaay': 'way', 'psssst': 'past','mooood': 'mood', 'usuallly': 'usually','ahhhh': 'aah', 'yasss': 'yes', 'lossses': 'losses', 'nevaaaa': 'never', 'stilllovewegmans': 'still love wegmans', 'mannn': 'man', 'awwww': 'aww', 'fukkkk': 'fuck', 'exciiited': 'excited', 'ahhh': 'aah', 
                'allll': 'all', 'outdooors': 'outdoors', 'uuugh': 'ugh', 'aaah': 'ahh', 'blesssss': 'bless', 'maaaybe': 'may be', 'dayssss':'days', 'bruhhh': 'bruh', 'itttttttt': 'it', 'waaaaaaaaaaaay': 'way', 'raaaaah': 'rah','eatwelllivewellku': 'eat well live well','yyyyes': 'yes', 'goooooooo': 'go', 'alllllllllll': 'all', 'aaand': 'and','ahhhhh': 'aah', 'hmmmm': 'hmm', 'fuuuuccckkkkkk': 'fuck', 'thoughtttt': 'thought', 'buttt': 'but', 'storessss': 'stores', 'goooo': 'go', 'ohhhh': 'oh', 'thennnn': 'then', 'whyyy': 'why', 'dumbbb': 'dumb', 'fuuuuuck': 'fuck', 'alllll': 'all','wooeee': 'woe', 'boooooo': 'boo', 'nahhhhh': 'nah', 'monthssss': 'months', 'shoooing': 'shooing', 'massshootings': 'mass shootings', 'amerikkka': 'amerika', 'hiiighly': 'highly', 'awww': 'aww', 'eeesh': 'esh', 'jooookeee': 'joke', 'realllllyyy': 'really',
                'okokokokokkkk': 'ok','aaah':'aah','aaahh':'aah', 'ahhh':'aah','ahhhh':'aah','ahhhhh':'aah','byeee': 'bye', 'yeahhh': 'yeah', 'neeeda': 'need', 'shiiiiiiz': 'shit', 'buuuut': 'but', 'hateeeee': 'hate', 'otttt': 'ot', 'hnnng': 'hang', 'aaaaugh': 'ugh', 'partayyyyyy': 'party', 'heatedddd': 'heated', 'aayyy': 'aay', 'buuutttt': 'but', 'trashhhh': 'trash', 'waaaay': 'way', 'whaaat': 'what', 'uhhh': 'ugh','sweetoooo': 'sweet', 'fuuuuq': 'fuck', 'honeyyy': 'honey', 'aaaand': 'and', 'teeeellll': 'tell', 'coolinnnnnn': 'colin', 'businessnewzzz': 'business news', 'grrr': 'grr','hellllllo': 'hello', 'pleaseee': 'please', 'whaaaaa': 'wha', 'whhhyyyyy': 'why', 'ewwwwww': 'eww','grrrr': 'grr', 'lolll': 'lol', 'bihhhhhh': 'bih', 'yesssss': 'yes', 'arghhhh': 'argh', 'lovellloves': 'love','pfffft': 'pft', 'lessshoppingmorewriting': 'less shopping more writing', 'workswonderfullly': 'works wonderfully', 'grocerybusinessservice': 'grocery business service', 'litttttt': 'lit', 'juuuuust': 'just', 'couchhhhhh': 'couch', 'insteaddddd': 'instead', 'innnnteresting': 'interesting', 'waaaaah': 'wah', 'fuuuuuuck': 'fuck', 'gawwwwd': 'gawd', 'sooooo': 'so', 'daaang': 'dang', 'hellllllllllp': 'help', 'sawww': 'saw', 'booo': 'boo', 'yaaaasssss': 'yes', 'grrrrrr': 'grr', 'livveee': 'live', 'starvingggg': 'starving','worthlessshopper': 'worthless hopper', 'pleaseeee': 'please', 'terrrrible': 'terrible', 'wrrrrr': 'wrr', 'neeeeeeed': 'need','gurrrrrr': 'grr','outtttt': 'out', 'nowzzz': 'now', 'maaaan': 'man', 'hungryyyyy': 'hungry', 'reeeeeeally': 'really', 'wheee': 'whee', 'groooooans': 'groan', 'errmaaaagaaaahdddd': 'omg', 'booooooooooooo': 'boo', 'haaaaaaaaaaaaaaaaaaaaaaaaaaate': 'hate','frannnd': 'frand', 'hmmmmmm': 'hm', 'arrreeeeee': 'are', 'laaawd': 'loud', 
                'foodddddddddd': 'food','coffeee':'coffee', 'lesssocietymoreworders': 'less society more worders', 'unluckyyyyy': 'unlucky', 'naaahr': 'nah', 'sweeeeet': 'sweet', 'ahhhhhh': 'ahh', 'eeep': 'eep', 'baaaare': 'bare', 'geeky0001': 'geky01', 'loooooook': 'lok', 'dribbble': 'drible', 'm9mk8888m9':'m9mk8m9', 'yassss': 'yas', 'jeeez': 'jeez', 'miiighty': 'mighty', 'ummmm': 'umm', 'moooostly': 'mostly', 'whaaaaaa': 'wha', 'ooooppssss': 'oops', 'daaaa': 'daa', 'woooffffff': 'woof', 'wannna': 'wana', 'selllig': 'selig', 'convenienttentttttt': 'convenientent', 'speeedy': 'speedy', 'whaaaaat': 'what', 'yaaaas': 'yes', 'hnnnnnng': 'hang','beeeeeeeeeach': 'beach', 'strollling': 'strolling', 'dddribbble': 'drible','wiiggggg': 'wig',  'reeeeeal': 'real', 'dolll': 'doll', 'uppp': 'up', 'reallllllllly': 'really', 'loveee': 'love', 'girrrrl': 'girl', 'cmoooooon': 'come on', 'neeeeed': 'need', 'fuuuuuuuuuuuuuuuuuuuuuuuckkkk': 'fuck','businessscript': 'business script', 'ehhh': 'eh', 'cluuutch': 'clutch', 'kiiinda': 'kinda','reeeaaaaally': 'really', 'eeeugh': 'enough', 'welcummm': 'welcome', 'embarsssed': 'embarrassed', 'buuuuutttt': 'but','looovvve': 'love', 'weee': 'we', 'ooohhhh': 'oh', 'unveilingthebusinessstories': 'unveilingthebusinestories',  'guuuuhyperlink': 'hyperlink','riiising': 'rising', 'biittttccchhheeeessss': 'bitch', 'whewwwwwwwwww': 'whew', 'whaaatttt': 'what', 'stilll': 'still', 'huuuuge': 'huge', 'suuuper': 'super', 'alllllllll': 'all', 'mannnn': 'man', 'pffft': 'pft', 'reallly': 'really', 'whuuuuuuh': 'whuh', 'suckssss': 'sucks', 'awonderfullly': 'wonderfully', 'staaaaaarve': 'starve', 'cuuute': 'cute', 'ayyyyyyy': 'ay', 'yaaassssszzzzz': 'yes', 'goooees': 'goes','toooooo': 'to', 'bleeeeh': 'bleh', 'harrrrrd': 'hard', 'maaaaad': 'mad', 'lazyyyyyy': 'lazy', 'uuuuummm': 'umm','bweakfaaaaaaaast': 'bweakfast', 'againnnnnn': 'again', 'hrmmmm': 'hrm', 'hooopefully': 'hopefuly', 'psssttt': 'pst','swearrrrr': 'swear', 'weeeeeek': 'week', 'brahhhh': 'brah', 'checkkkk': 'check','everrrrr': 'ever', 'annnnnnnd': 'and', 'suuuucks': 'sucks', 'atttorney': 'attorney', 'peanutbuttter': 'peanut butter', 'ayyy': 'ay',  'wtffff': 'wtf', 'tttthhhhhiiissssss': 'this', 'hateee': 'hate', 'savedddd': 'saved','squeeeee': 'squee',  'oooouuuu': 'ou', 'naaaaa': 'na', 'weeee': 'we', 'yessss': 'yes', 'yeaaaa': 'yes', 'homeacccess': 'home access', 'timberrr':'timber', 'tahdayyy': 'today', 'yeees': 'yes', 'stayinnn': 'staying', 'aliveee': 'alive', 'heeeeeere': 'here', 'gr000oceries': 'groceries', 'taaaagetherrr': 'together', 'appp': 'app', 'sighhh': 'sigh', 'wooohoo': 'woohoo', 'winterrrr': 'winter', 'friiiendssss': 'friends', 'ooooof': 'of', 'yuuup': 'yes', 'everythinggg': 'everything', 'thankkk': 'thank', 'youuu': 'you', 'aaaannndddd': 'and', 'babaaaay': 'baby', 'ayeee': 'aye', 'babiesss': 'babies', 'yaaas': 'yes', 'coinnn': 'coin', 'nevvvaaaa': 'never','aaaack': 'ack', 'mmmlove': 'love', 'dayyyy': 'day', 'cribmusss': 'cribmus', 'employeeengagement': 'employee engagement', 'diseaseeee': 'disease', 'beyonddd': 'beyond', 'jeeeeesus': 'jesus', 'coffeeeeee': 'coffee', 'wwasss': 'was', 'alriiight': 'alright', 'tieeeeeeed': 'tied','hellooooo': 'hello', 'reeeaaalllll': 'real', 'whoooo': 'who', 'hulll': 'hull', 'whhyyyyy': 'why', 'oooooold': 'old','orrrr': 'or', 'weeeek': 'week', 'dayyyyy': 'day', 'ohhhyeaaahhh': 'oh yeah', 'helloooo': 'hello', 'yallll': 'you all','agghhhhhhhhhhhhhhhhhhhhhhhhhhh': 'argh', 'agghhhhhhhhhhhhhhhhhhhhhhhhhhhhh': 'argh', 'finallly': 'finally','adressses': 'addresses', 'reallllly': 'really', 'omggg': 'omg', 'yaaaaa': 'ya', 'laggg': 'lag', 'helllooooee': 'hello', 'hellllooooee': 'hello', 'hungryyy': 'hungry', 'apppayments': 'apayments', 'littt': 'lit', 'suuuuuuuuper': 'super', 'feeees': 'fees', 'bihhhh': 'bih', 'gawddd': 'gawd', 'allllll': 'all', 'deeeply': 'deeply', 'fullfilllment': 'fulfilment', 'pfff': 'pf', 'weirdddd': 'weird', 'nowwwwwww': 'now', 'heyyyy': 'hey','luuuuucky': 'lucky', 'haaand': 'hand', 'nowwwww': 'now',  'heeelp': 'help', 'shppper': 'shopper', 'mooost': 'most', 'fooods': 'foods', 'girlllll': 'girl','whatttt': 'what', 'factttt': 'fact', 'yaaaassssss': 'yes', 'worrry': 'worry', 'buuutt': 'but', 'andddd': 'and','groooound': 'ground', 'commmunication': 'communication', 'biggg': 'big', 'geeeezus': 'jeesus',              'woooo': 'woo', 'paaaranoid': 'paranoid', 'wuaaaaaaaaaaaa': 'wua', 'yerrrrrr': 'yer', 'craaaap': 'crap','weeeeeed': 'weed', 'daaaays': 'days', 'runnning': 'running', 'eeech': 'ech', 'everrr': 'ever', 'quinnnnnnn': 'quin','yummm': 'yum', 'goood': 'god',  'okaaaaay': 'okay', 'weeeeeeeee': 'we', 'chilll': 'chill', 'yayyy': 'yay', 'realllly': 'really', 'oooof': 'of', 'wayyyyy': 'way', 'booooo': 'boo', 'yyyyeesssssssss': 'yes', 'stafff': 'staff', 'ooooooookay': 'okay', 'suppposed': 'supposed','thanksssss': 'thanks', 'boyyyyy': 'boy', 'ugguhughughghhhh': 'ugh', 'reaalllyy': 'really', 'tyyy': 'ty','yesss': 'yes', 'weeeird': 'weird', 'wheeeeeeee': 'whee', 'aieee': 'aie', 'pissssssed': 'pissed', 'anddddd': 'and','errrr': 'err', 'freeeeeee': 'free', 'dayzzzz': 'days', 'killerrrr': 'killer', 'piiised': 'pissed', 'wooooop': 'woop', 'localbusinessservices': 'local business services', 'happyhmmm': 'happy','housssseeee': 'house', 'gaaaaah': 'gah', 'sbappp': 'sbap', 'scurrred': 'scurred', 'messsages': 'messages', 'whaaaat': 'what', 'soooon': 'soon', 'theee': 'the', 'villlage': 'village', 'woooow': 'wow','chiliiii': 'chili', 'uhhmmmm': 'uhm', 'itwonderfullly': 'it wonderfully','rulllessssss': 'rules', 'aaaannnnnd': 'and', 'ewwwhyperlink': 'hyperlink', 'sammeee': 'same', 'geeez': 'geez', 'wayyyyyyy': 'way', 'eatwelllivewell': 'eat well live well', 'tooo': 'too', 'nopeeee': 'nope', 'aswonderfullly': 'as wonderfully', 'aaaaaaaaallllll': 'all', 'resuppply': 'resupply', 'yeahhhhh': 'yeah', 'lesss': 'less', 'innnnn': 'in','shppping': 'shipping', 'yeaaaah': 'yeah', 'whyyyyy': 'why','ooops': 'oops', 'neoowww': 'neow', 'seeems': 'seems', 'cluuuuuuutch': 'clutch', 'hellllll': 'hell', 'yaaaaaa': 'ya', 'aaahh': 'aah', 'byyyyyeeeeee': 'bye', 'pleaseeeeeee': 'please', 'maaaaan': 'man','lollll': 'lol', 'caaaat': 'cat', 'foood': 'food', 'faaaaaave': 'favourite', 'knowwww': 'know', 'weeeks': 'weeks', 'yaaa': 'ya','reeeeaaaalllly': 'really', 'themmm': 'them', 'gooooooood': 'good','ssshhhh': 'shh', 'waaaawaaaaawaaaaa': 'wow', 'businessschools': 'business schools', 'locallly': 'localy', 'boooooy': 'boy', 'plzzzzz': 'plz', 'realllyyyy': 'really', 'lmdaoooo': 'lmdao', 'daaaaaam': 'damn', 'woohooo': 'woohoo', 'whaattt': 'what','truuuuu': 'true', 'whaaaaaaat': 'what', 'meeee': 'me', 'reeeeeallly': 'really', 'ffffuuuuccccckkkkkkkkkkk': 'fuck', 'winnnning': 'winning', 'pppl': 'ppl', 'siiiigh': 'sigh', 'thurrr': 'thur', 'hopefullly': 'hopefully', 'lettting': 'letting', 'reaaaaaaally': 'really', 'buisnesssupport': 'buisness support', 'bombbbb': 'bomb','wuuut': 'wut', 'fuuu': 'fuu', 'trafffic': 'traffic', 'butttt': 'but', 'coffeee': 'coffee', 'maaamm': 'mam', 'reeeaally': 'really', 'gooo': 'go', 'sammeeee': 'same', 'swoooooon': 'swon', 'todaaaaaaay': 'today', 'nccc': 'nc','uggg': 'ugh', 'hellllllp': 'help', 'grrrrrrr': 'grr', 'shoooot': 'shot', 'fffuzzy': 'fuzzy', 'gettting': 'getting','telll': 'tell', 'maaaaybe': 'may be', 'duuuuude': 'dude', 'dooooo': 'do', 'weyyy': 'wey', 'boycotttrumppressconferences': 'boycotrumpresconferences','nowww': 'now', 'hooraaaay': 'hooray',  'woohoooo': 'woohoo', 'hhhmmm': 'hmm', 'yummmmmm': 'yum', 'wellllllll': 'well', 'woahhh': 'whoa', 'buuuuusssssinesssss': 'business','muuuumm': 'mum', 'girlll': 'girl', 'uuuuugh': 'ugh', 'thiiis': 'this', 'wayyyy': 'way', 'ateeee': 'ate', 'ohhhhhh': 'oh', 'ahhhhhhhhhhh': 'aah','saaaaammmmeee': 'same', 'saaammmmeee': 'same', 'infinittyyyyyy': 'infinity','welppp': 'welp', 'hellooooooo': 'hello', 'tryinggg': 'trying', 'roaaaaarsome': 'roarsome', 'cooo': 'co', 'oookies': 'okay','ittttttt': 'it', 'willl': 'will', 'neededdddddd': 'needed', 'haaay': 'hay', 'daysss': 'days', 'hahahhhahah': 'haha','oooph': 'oph', 'youlllive': 'you will live', 'defffff': 'def', 'bitttttt': 'bit', 'ideaaaa': 'idea', 'waitttt': 'wait','dangggggggggg': 'dang', 'lmaooimfatboyandwillstarveooooooooooooooooooo': 'lmao i am fat boy and wil starve', 'aaaannnnnndddddd': 'and', 'cevicheee': 'ceviche', 'tostadassss': 'tostadas', 'boooooooox': 'box', 'yaaaay': 'yay', 'guyssss': 'guys', 'aaaassssss': 'as', 'allllllaaaat': 'alat', 'waaaaayyyyy': 'way', 'coooooookies': 'cookies', 'folksss': 'folks', 'meee': 'me', 'yuuuup': 'yup', 'daaark': 'dark', 'sorryyy': 'sorry', 'concepttttt': 'concept', 'booosted': 'boosted','threeeee': 'three', 'grrrrr': 'grr','starrrrrrrrrrving': 'starving',  'baybayyyy': 'baby', 'aaaaafffffff': 'aaf', 'maxxxx': 'max', 'waaaaaaaah': 'wah', 'arghhh': 'argh',  'flosssssss': 'floss',  'wayyy': 'way', 'burgesss': 'burges', 'wahhh': 'wah', 'maaam': 'mam', 'brrrr': 'brr', 'yippeeeee': 'yippee', 'loool': 'lol', 'ooohh': 'oh', 'loooooool': 'lol', 'donneeee': 'done', 'ewwww': 'eww', 'everrrrrr': 'ever', 'looot': 'lot', 'deliverrrry': 'delivery', 'isss': 'is', 'orrrrrr': 'or', 'duuuude': 'dude', 'ftwwwwww': 'ftw', 'hallllp': 'help', 'fuuuuck': 'fuck', 'yaaaaas': 'yes','businessscience': 'business science','myyy': 'my', 'successstory': 'sucess story', 'fffffffuck': 'fuck', 'thisssssss': 'this', 'ahhhhhhh': 'aah', 'yaaaaasss': 'yes', 'tthhhe': 'the', 'donttouchmeee': 'dont touch me',  'jeeeeeze': 'jeeze', 'suuuuch': 'such', 'twattttt': 'twat', 'tomorrrow': 'tomorrow', 'aaalllll': 'all','gooooaaaaaalll': 'goal', 'heeeeeeere': 'here', 'weeeeeee': 'we', 'gooooo': 'go', 'firrre': 'fire', 'chainssss': 'chains', 'whhhaaatttt': 'what', 'yaaasss': 'yes', 'notttt': 'not', 'haaaaaaaate': 'hate','hungeeeee': 'huge', 'waaaaa': 'wah', 'wheeeeee': 'whee', 'milllennials': 'millennial', 'aghhh': 'agh', 'errrrr': 'err', 'bestttt': 'best', 'buuuuuut': 'but', 'hmmmph': 'hmph', 'maaaaaaaan': 'man', 'buuuuut': 'but', 'guuuuurrrllll': 'girl', 'heyyy': 'hey', 'ummmmmmm': 'umm', 
                'yassssss': 'yes', 'fuckkk': 'fuck', 'arhhh': 'arh', 'hooo': 'ho', 'itttttt': 'it', 'boooo': 'boo', 'zzzzz': 'z', 
                'nahhhh': 'nah','guyssss':'guys','damnnnn':'damn', 'grrr':'grr','grrrr':'grr','grrrrrrrr': 'grr', 'mmmaybe': 'may be', 'deeeeets': 'details', 'waaaaaay': 'way', 'knowwwwww': 'know', 'hollaaaaaaa': 'hola', 'exxxtra': 'extra', 'neeeed': 'need', 'yaaaassss': 'yas', 'seamlessshopping': 'seamleshoping','plsss': 'please', 'hnnnnngggggg': 'hang', 'retweeet': 'retweet', 'retweeetplease': 'retweet please', 'tyyyyy': 'ty','yaaaaassss': 'yes', 'aahhh': 'aah','yupppp': 'yup', 'damnnnnnnn': 'damn','successstories': 'success stories', 'nnnreal': 'unreal', 'reggggular': 'regular', 'screeeam': 'scream', 'stahhpppp': 'stop', 'girrrrllllllllll': 'girl', 'mooooooooooon': 'moon', 'ouuuuuuuchhhh': 'ouch', 'yoooo': 'yo', 'lessstress': 'less stress', 'positivevibezzz': 'positive vibes', 'oookkkkayyy': 'okay', 'breeezyy': 'breezy', 'wisssshh': 'wish', 'foooood': 'food', 'oooooooookay': 'okay','thaaanks': 'thanks', 'loook': 'look', 'paaaarrrrrty': 'party','hollaaaa': 'hola', 'suuuuuper': 'super', 'hateeee': 'hate', 'gaaaaahhhhhh': 'gah','callling': 'calling','honestbeeeeee': 'honestbee', 'hoooonest': 'honest', 'neeeeeeeeeever': 'never', 'ricccch': 'rich', 'juuuuuuust': 'just', 'whaaattt': 'what', 'whyyyyyy': 'why','happyxxx': 'happy','ayyye': 'aye', 'hardddd': 'hard', 'pleeeeeeease': 'please','suuuuucks': 'sucks', 'shhhhh': 'shh', 'naaaaaaaaah': 'nah', 'heeey': 'hey', 'yooooo': 'yo','wooooooooorse': 'worse', 'ittttt': 'it', 'buuut': 'but', 'succkssss': 'sucks', 'slowwwwlyyyy': 'slowly','yaaassss': 'yes', 'annnnnd': 'and', 'herrreeee': 'here', 'yeaaaaaah': 'yeah', 'murderrrr': 'murder','yeaaaaaaaaaah': 'yeah', 'aaaany': 'any', 'uppppp': 'up', 'fuuun': 'fun', 'tadaaa': 'tada', 
                'hufffff': 'huff', 'juuust': 'just','finnne': 'fine','errr': 'err', 'odddeliverydrivercomplaints': 'odd delivery driver complaints', 'eeek': 'eek', 'girrrrrl': 'girl', 'hmmmmmmm': 'hmm', 'onlyyyyyyy': 'only', 'guysssssss': 'guys','lotttttt': 'lot', 'ohhhhh': 'oh', 'ruuuns': 'runs', 'weeeekend': 'weekend', 'shoppping': 'shopping', 'verrry': 'very', 'fooooooood': 'food', 'tooooooo': 'too', 'reeeeally': 'really','zoooo': 'zoo', 'uhhhh': 'ugh', 'sorrryyyy': 'sorry', 'ssshhh': 'shh', 'embarrassmentummm': 'embarassment', 'plssssssss': 'please', 'todayyyy': 'today', 'neverrrr': 'never', 'craaazy': 'crazy', 'annnd': 'and', 'boooomb': 'bomb','aliiive': 'alive', 'ecologicallly': 'ecologicaly', 'nooo': 'no', 'waaaait': 'wait', 'tooooo': 'too', 'ohhhhhhh': 'oh', 'whyyyy': 'why', 'pleeaaaassseeee': 'please', 'fuuuudge': 'fudge','shiddddd': 'shid', 'biiig': 'big', 'yooooooo': 'yo', 'coldddddddddd': 'cold', 'mighttttt': 'might','freeeee': 'free', 'pleaseeeeee': 'please', 'helll': 'hell','mmkkayyy': 'i am okay', 'reeeaaalll': 'real','bxtchhhhhhh': 'bitch', 'eeeeek': 'eek', 'ooooowweeeeee': 'owe','toooooooo': 'too', 'retweee': 'retweet', 'daaayyyysss': 'days', 'hmmmmmmmmmmm': 'hmm', 'heeeeere': 'here', 'fineeee': 'fine', 'naurrr': 'naur', 'liviiddddd': 'livid', 'looool': 'lol', 'stokedddd': 'stoked', 'pleeeease': 'please', 'arghhhhhh': 'argh', 'especiallly': 'especialy', 'supppper': 'super', 'ummmmmm': 'umm', 'seeee': 'see', 'wtfff': 'wtf',  'slowwwwwservice': 'slow service', 'suuuuuuper': 'super', 'whhhaaaaaaaa': 'whaa', 'soooers': 'soers','eeeeeeeeeekkkkkkkk': 'eek', 'don888': 'don8', 'extreeeemly': 'extremely','saaame': 'same', 'booooy': 'boy', 'ooof': 'of', 'bitchhh': 'bitch', 'babbieeeee': 'baby', 'refuseeee': 'refuse', 'jooooooooooke': 'joke', 'juskooo': 'jusko', 'vaxxxed': 'vaxed', 'loooow': 'low', 'slowwwww': 'slow', 'waaaayyyyy': 'way','businesssuccess': 'business success', 'naaaaaaa': 'na', 'goooooooooooooo': 'go', 'fuuuuuuuck': 'fuck', 'shoppinggg': 'shopping', 'bihhh': 'bih', 'pshhh': 'psh', 'saaaaaame': 'same', 'faaaaaaccckkk': 'fuck', 'employeeengagment': 'employee engagement', 'ooop': 'oops', 'sighhhhhhhhhhhhhhhhh': 'sigh', 'howwwww': 'how', 'ooooold': 'old', 'ghhhhggh': 'ghgh', 'shitttt': 'shit', 'tfff': 'tf', 'saaaaame': 'same', 'whaaaaatttt': 'what', 'yeeeeaah': 'yeah', 'seeet': 'set', 'yeee': 'ye', 'uuuugggg': 'ugh',  'pleaseeeee': 'please', 'tipzzzzz': 'tips', 'aarrrrrrrrghhhhhhh': 'argh', 'foreseennnnn': 'foreseen', 'yooouuu': 'you', 'ayyyyyy': 'ay', 'sucksss': 'sucks', 'screeeeeams': 'screams',  'mamoy': 'mamoy',  'whoooooof': 'whoof', 'chillle': 'chile', 'buttttt': 'but', 'actuallly': 'actually', 'wooot': 'woot', 'businessstandard': 'business standard', 'idcccc': 'idc', 'fulllllll': 'full', 'seeeeesh': 'sesh', 'handddd': 'hand', 'smhhh': 'smh', 'sodaaaa': 'soda', 'smallbusinesssupport': 'small busines support','nervessssssss': 'nerves', 'snowww': 'snow', 'ssssssssssoooo': 'so', 'biiit': 'bit', 'wooooooooooo': 'woo', 'swwwwing': 'swing', 'shittty': 'shity', 'yyyy': 'y', 'maaaann': 'man', 'tippppsss': 'tips', 'neeeeeeeeeeeds': 'needs', 'shoppinggggg': 'shopping', 'arghhhhh': 'argh', 'listennn': 'listen', 'neeeever': 'never', 'eeeep': 'eep', 'tooold': 'told', 'bahhh': 'bah', 'sneeeeeeezers': 'sneezers', 'lovexxx': 'love', 'yeeepppp': 'yep', 'aaamazzzing': 'amazing', 'fuuuuuuuuuuuuck': 'fuck','omfggg': 'omfg', 'monthsssss': 'months', 'maaaaaaaaay': 'may', 'arrrrghhh': 'argh', 'dyinggggg': 'dying', 'muuuuuuuuch': 'much', 'wheeew': 'whew','springgggg': 'spring', 'cmonnnnnn': 'cmon', 'thaaat': 'that', 'jeeeez': 'jeez', 'nawww': 'naw', 'shhhh': 'shh', 'waaaaahhh': 'wah', 'thatttt': 'that', 'clutchhhhhhhh': 'clutch', 'squeeee': 'squee', 'maaaaaan': 'man', 'eatttt': 'eat', 'woweee': 'wowe', 'plzzzz': 'plz', 'babyyyyy': 'baby', 'crucialllll': 'crucial', 'beekeeeper': 'beekeeper', 'beeee': 'be', 'morningggg': 'morning', 'stopppppo': 'stop', 'itsalllocal': 'its a local', 'hmrsss': 'hmrs','successsee': 'success','burrrr': 'burr', 'wheeee': 'whee', 'anyoneeee': 'anyone', 'annndd': 'and', 'businesss': 'business', 
                'incrennnible': 'incredible','lmaooo':'lmao', 'helloooo':'hello','gaaaah': 'gah', 'blowssssssss': 'blows', 'veeeeeggggg': 'veg', 'thiissss': 'this', 'eccc': 'ecc', 'seee': 'see','nahhhhhh': 'nah', 'caaaaasually': 'casually', 'thissss': 'this', 'wellllll': 'well', 'sayinnggg': 'saying', 'dinnerrrr': 'dinner', 'ughhh': 'ugh', 'coffeeeee': 'coffee', 'apppreciate': 'appreciate', 'fooddddddd': 'food', 'waaaa': 'waa', 'blacklivesmattters': 'black lives matters', 'pleaseeeeeeeee': 'please', 'haaaa': 'haa','bagsss': 'bags', 'aiyaaa': 'aiya', 'sixxxxxx': 'six', 'weeeeekssssss': 'weeks', 'aaaaaaaaaaaaaaahhhhhh': 'aah','suuure': 'sure', 'youuuu': 'you', 'freestufffriday': 'free stuff friday', 'bammmmmm': 'bam', 'ewww': 'eew', 'sisss': 'sis', 'yeesss': 'yes', 'sweeeeeeeet': 'sweet', 'girllll': 'girl', 'suuuuck': 'suck', 'wooooooah': 'woah', 'sayweee': 'say we', 'yoooooo': 'yo', 'hellllla': 'hell', 'eeegads': 'egad', 'lmfaaooo': 'lmfao', 'hhhhhg': 'hg', 'fooooooodddd': 'food', 'fooooood': 'food', 'tttil': 'till', 'sayyyy': 'say', 'booooooo': 'boo','faaaar': 'far', 'peopleeeeeeee': 'people', 'auuugh': 'ugh', 'fuuuck': 'fuck', 'sirrr': 'sir', 'plzzz': 'plz', 'yummmm': 'yum', 'sleep': 'sleep', 'alsooo': 'also', 'reallllllly': 'really', 'nahhhhhhting': 'nothing', 'wayyyyyyyy': 'way', 'wheeeeeeeeee': 'whee', 'niiice': 'nice', 'shoooo': 'shoo', 'yeaaah': 'yeah', 'bwahhhahaah': 'haha', 'hahaaaa': 'haha', 'ordererrr': 'orderer', 'behhh': 'beh', 'plsssss': 'pls', 'neeed': 'ned', 'pffffftt': 'pft', 'choooooosing': 'chosing', 'byeeeeeeee': 'bye', 'welllll': 'well', 'naaaame': 'name', 'strugggllleeeee': 'struggle', 'yeahhhh': 'yeah', 'waayyy': 'way', 'aaaw': 'aww', 'yummmmmmmmmmmm': 'yum', 'nahhh': 'nah', 'silaaaaaa': 'sila', 'aaaaahhhhh': 'aah', 'bayyyy': 'bay', 
                'saaammee': 'same','alll':'all', 'grrr':'grr','yummm':'yummy','yeees':'yes','yasss':'yes','yassss':'yes','whyyy':'why','whaaat':'what','lolll':'lol','weee':'we', 'weeee':'we','tooo':'too','tyyy':'ty','uuugh':'ugh', 'sooo':'so','suuuuper': 'super', 'whyyyyyyy': 'why', 'reaaaaaally': 'really', 'fuccc': 'fuck', 'neverrr': 'never', 'ffff': 'f', 'maaan': 'man', 'asssist': 'assist', 'uuuuu': 'u', 'foreverrrrrer': 'forever',  'aaaaaah': 'aah', 'hellooooooooooo': 'hello', 'mmmmmmmrrrrrrrrrrrr': 'mr',  'mmmrrrrrrrrrrooooooooooorrrm': 'mrorm', 'feee': 'fee', 'freee': 'fre', 'stressss': 'stress', 'yeeeeee': 'yee', 'deeeeerrrrrp': 'derp', 'wooohoooo': 'woohoo', 'mehhhh': 'meh', 'bahhhh': 'bah', 'chillinnnn': 'chilling', 'whattt': 'what', 'lmaooooo': 'lmao', 'daaaaamn': 'damn', 'lolllll': 'lol','psssssst': 'pst', 'carrrrrr': 'car', 'paaaain': 'pain', 'reeeeeee': 're', 'madeeeee': 'made', 'oooow': 'oaw', 'wooooo': 'woo', 'ahhhhhhhhhgg': 'ahg',  'blehhhhh': 'bleh', 'teeeeeny': 'tiny', 'carbzzzz': 'carbz', 'efffing': 'effing', 'kakraaa': 'kakra', 'hoooooo': 'ho', 'jeeeeez': 'jeez', 'hahaaa': 'haha', 'suuuuuuuuure': 'sure', 'hummm': 'hum', 'myoooosive': 'myosive', 'prrrrobably': 'probably', 'waiiit': 'wait','fuuuuuuucked': 'fucked', 'thanksgiviiing': 'thanksgiving', 'waaaant': 'want', 'girrrlll': 'girl', 'gaaaame': 'game', 'whewww': 'whew', 'awwwwkward': 'awkward', 'godddddd': 'god', 'whoooooo': 'who', 'dopppeee': 'dope', 'looolll': 'lol', 'ammmmiwrongggggg': 'am i wrong', 'weeek': 'week', 'yeeeeahhh': 'yeah', 'knowww': 'know', 'nottt': 'not', '4evaaaa': '4eva', 'wiiild': 'wild', 'wooo': 'woo', 'yummmmmmmm': 'yum','aaaall': 'all', 'herreeeee': 'here', 'seasonnnnnn': 'season', 'ummmmmmmm': 'umm', 'goootcha': 'gotcha', 'oooops': 'oops', 'yeeeees': 'yes', 'sheeeeeeeee': 'she', 'mmmfph': 'mfph', 'everrrr': 'ever', 'stufff': 'stuf', 'fkkkk': 'fuck', 'eliteeeeeeee': 'elite', 'maybeeee': 'maybe', 'brrrrr': 'brr', 'yyeesss': 'yes', 'waaaayyy': 'way', 'reaallly': 'really', 'lmaaao': 'lmao', 'eyyy': 'eyy', 'givingggg': 'giving', 'toooo': 'too', 'hiii': 'hi', 'yeeeeeeeees': 'yes', 'arrrgh': 'argh', 'fuuuccccckkk': 'fuck', 'covidvacccine': 'covid vaccine', 'mmmay': 'may', 'wellll': 'well', 'yeeeeesss': 'yes', 'laaaaa': 'la', 'sameeee': 'same', 'goooood': 'good', 'afternoonnnn': 'afternoon', 'fffb': 'fb', 'aaaccedent': 'accident', 'clutcchhhhh': 'clutch', 'whennnnnn': 'when', 'reeeeeeeally': 'really', 'rigghhhttttt': 'right', 'ahhhhhhhh': 'aah','walkkk': 'walk', 'phewwww': 'phew', 'overrr': 'over', 'fuuuuuuuuck': 'fuck', 'aaaah': 'aah', 'classsss': 'class', 'maaaaaybe': 'may be', 'daaaamn': 'damn', 'businessses': 'busineses', 'hooooo': 'hoo', 'whooo': 'who', 'onlineshoppping': 'online shopping','hiswonderfullly': 'his wonderfully', 'paymisseee': 'paymise', 'hoooo': 'ho', 'orrr': 'or', 'waaaaaaah': 'wah', 'aaahhhhhh': 'aah', 'caaalllllll': 'call', 'reeeally': 'really', 'ooookay': 'okay', 'juuuuuuuuuuust': 'just', 'earlyyyy': 'early', 'fuuuccckkk': 'fuck', 'caaaant': 'cannot', 'maaaannnnnn': 'man', 'uuuuuuuu': 'u', 'boooring': 'boring', 'downnnn': 'down', 'cuteee': 'cute', 'gaaaarooooossssssss': 'garos',  'buttttttttt': 'but', 'racccisssttt': 'racist', 'baaaaaaby': 'baby', 'orrrrrrr': 'or', 'reallyyy': 'really', 'aaaanndd': 'and', 'oohhhh': 'oh','anddd': 'and', 'happychildlessspinster': 'happy childless spinster', 'goooooo': 'go', 'baaaack': 'back','succckkks': 'sucks','whaaaaaaaat': 'what', 'thruuuuuuuuuuu': 'thru', 'whooot': 'whot', 'haaaaaaaaate': 'hate', 'nervousssss': 'nervous', 'heeeeeeeelpppppppppp': 'help', 'mannnnnn': 'man', 'oooooooook': 'ok', 'yussss': 'yes', 'bigggggg': 'big', 'igrrr': 'igr', 'botttom': 'bottom', 'muuurrr': 'mur', 'suuucks': 'sucks','juuuust': 'just', 'awwwhyperlink': 'hyperlink', 'speeeedy': 'speedy', 'uuuf': 'uff', 'crazzzzyyyy': 'crazy', 'aaahhhh': 'aah', 'businesssolutions': 'business solutions', 'teeesst': 'test', 'yeeeah': 'yeah', 'gimmme': 'give me', 'thewonderfullly': 'the wonderfully', 'ffffffuck': 'fuck', 'baybeeeee': 'baby', 'backkk': 'back', 'sweetttttt': 'sweet', 'myhhhh': 'myh', 'babyyyy': 'baby', 'todayyyyyy': 'today', 'laaaa': 'la', 'fulllockdown': 'full lockdown', 'goddddd': 'god', 'maaaannn': 'man', 'uuuugh': 'ugh', 'tierrrrrrrrr': 'tier', 'sleeeeeping': 'sleeping', 'sunnnybeemarket': 'sunybee market', 'eeevvveeerrr': 'ever', 'haaarddddddd': 'hard', 'saaaame': 'same', 'pffffizer': 'pfizer', 'mmmmicrobes': 'microbes', 'byeeee': 'bye', 'founddd': 'found', 'serviceeeee': 'service', 'wantttt': 'want', 'buyyyy': 'buy', 'meaaaaaaat': 'meat', 'mmmh': 'mmh', 'wahhhhh': 'wah', 'heeeere': 'here', 'thankssss': 'thanks', 'hummmer': 'humer', 'yeeeeeeeeeaaaaaahhhhhh': 'yeah', 'unhappyblesssss': 'unhappy bless', 'dooor': 'door', 'welll': 'well', 'focuuussssss': 'focus',  'hisss': 'his', 'babyyy': 'baby', 'whateverrrr': 'whatever', 'weeeeee': 'we', 'bzzzzzzz': 'bz', 'naaaa': 'na', 'cosssssss': 'coss','lifeeee': 'life', 'businesssolution': 'business solution', 'pleaseeeeeeee': 'please','yuuum': 'yum', 'buttttttttttt': 'but', 'pissedddd': 'pissed', 'geeesh': 'geesh', 'helloooooo': 'hello', 'fineeeee': 'fine','alllllll': 'all', 'againnnn': 'again', 'muuuuch': 'much', 'ewwwww': 'eww', 'buuuunch': 'bunch', 'freeaaaakk': 'freak', 'aaauugh': 'ugh', 'aaaaahh': 'aah', 'aaaaaaaahhh': 'aah', 'waaaayyyyyy': 'way','hereee': 'here', 'shhh': 'shh', 'gooood': 'good', 'todayyy': 'today', 'outttt': 'out', 'nervesssss': 'nerves', 'yuuuuup': 'yup', 'sheeee': 'she','buildiiiiiing': 'building', 'bankkkkk': 'bank', 'usssa': 'usa', 'weeeee': 'we', 'nohhh': 'noh','bessst': 'best', 'ughnnn': 'ugh', 'worlddddddddddd': 'world', 'yeeeaaaaaahhh': 'yeah', 'whewwww': 'whew', 'suuuuuuucks': 'sucks', 'uhhhm': 'uhm', 'wuuuuu': 'wu', 'aaaaayyyy': 'ayy','10000x': '10x', 'perioddddd': 'period', 'alsoooo': 'also', 'thiccc': 'thick', 'lezzgoooo': 'let go', 'ighttttt': 'ight', 'giiirl': 'girl', 'd7000': 'd70', 'xxxl': 'xl', 'q17775306': 'q175306','hhhhhhhhhh': 'h', '$mmmb': '$mb', 'dxxx': 'dx','000$': '0$','000pm': '0pm', '1119th': '19th', '1000k': '10k', 'appph': 'aph', '$misssadiemaeve': '$misadiemaeve','x10000': 'x10', '0xf8da8ecee025444ad8dcf065ae09cffcd55c6116': '0xf8da8ece0254ad8dcf065ae09cfcd5c616'
                   }
len(replace_list)
                    

1274

In [61]:
len(replace_list.keys())

1274

In [62]:
total_count = 0
rep_words = []
for i in tqdm(range(len(data))):
    count = 0
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in replace_list:
            count += 1
            rep_words.append(word)
    total_count += count
total_count   

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 225777.44it/s]


3265

In [63]:
for i in tqdm(replace_list):
    data.loc[:,'tweet'] = data.tweet.str.replace(i,replace_list[i])

 99%|██████████████████████████████████████▋| 1264/1274 [02:54<00:01,  7.53it/s]/var/folders/2k/fhkt65ls68nbd5xt0lj3sbww0000gn/T/ipykernel_59839/2053584832.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.loc[:,'tweet'] = data.tweet.str.replace(i,replace_list[i])
100%|███████████████████████████████████████| 1274/1274 [02:55<00:00,  7.24it/s]


In [64]:
print(count_link(data), count_i(data), count_my(data))

195618 218051 87709


### Replace slangs

In [65]:
slangs = {}
with open('../../Supplementary data/slangs.txt', 'r') as f:
    for i in f:
        for j in i.split(','):
            j = j.replace('{','')
            j = j.replace('}','')
            j = j.replace('\"','')
            a = j.split(':')[0].strip()
            b = j.split(':')[1].strip()
            slangs[a] = b
len(slangs)

167

In [66]:
count = 0
slang = []
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if word in slangs:
            count += 1
            slang.append(word)
count

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 220657.58it/s]


15992

In [67]:
def replace_slangs(text):
    return ' '.join([slangs.get(i.lower(),i.lower()) for i in text.split()])
data['tweet'] = data['tweet'].apply(lambda x : replace_slangs(x))
print(data['tweet'][5003])
print(data['tweet'][5006])

my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [68]:
print(count_link(data), count_i(data), count_my(data))

195618 218209 87739


### Find words representing coronavirus

In [69]:
def replace_corona(text):
    replace_list = {'coronavirus':'corona','covid19':'corona','covidiot':'corona idiot','covid':'corona'}
    words = text.split(' ')
    new_words = []
    for word in words:
        for i in replace_list:
            if i in word:
                word = word.replace(i, replace_list[i])
        new_words.append(word) 
    return ' '.join(new_words)
        
    

In [70]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x : replace_corona(x))

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 271052.16it/s]


In [71]:
words = ' '.join(data['tweet']).split()
dic_corona = {}
for word in words:
    if 'corona' in word:
        if word in dic_corona:
            dic_corona[word] += 1
        else:
            dic_corona[word] = 1

In [72]:
new_words = dic_corona.keys()

In [73]:
import wordninja
for word in new_words:
    print(word, wordninja.split(word))
    

corona ['corona']
coronas ['coronas']
coronacanada ['corona', 'canada']
corona2019 ['corona', '2019']
nocorona ['no', 'corona']
coronalockdown ['corona', 'lockdown']
coronaactnow ['corona', 'act', 'now']
coronacalendar ['corona', 'calendar']
highriskcorona ['high', 'risk', 'corona']
coronataskforce ['corona', 'taskforce']
coronacrisis ['corona', 'crisis']
coronaoutbreak ['corona', 'outbreak']
coronatx ['corona', 'tx']
coronapocalypse ['cor', 'on', 'apocalypse']
coronaab ['corona', 'ab']
coronadining ['corona', 'dining']
coronachat ['corona', 'chat']
coronaetiquette ['corona', 'etiquette']
coronaquarentinedays ['corona', 'qu', 'arent', 'in', 'e', 'days']
seattlecorona ['seattle', 'corona']
coronalife ['corona', 'life']
coronabahamas ['corona', 'bahamas']
coronafacts ['corona', 'facts']
coronaidiots ['corona', 'idiots']
therescoronainsidethere ['there', 's', 'corona', 'inside', 'there']
floridacorona ['florida', 'corona']
coronacooties ['corona', 'cooties']
businessduringcorona ['busines

In [74]:
print(count_link(data), count_i(data), count_my(data))

195618 218209 87739


### Divide connected words

In [75]:
#Number of connected words
connect_words =[]
for i in tqdm(range(len(data))):
    words = data['tweet'][i].split(' ')
    for word in words:
        if len(word) >12 and word not in connect_words:
            connect_words.append(word)
len(connect_words)

100%|███████████████████████████████| 368413/368413 [00:03<00:00, 116644.05it/s]


15018

In [76]:
import wordninja
def split_words(text):
    words = text.split()
    new_words = []
    for word in words:
        if len(word) > 12:
            new_words.extend(wordninja.split(word))
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [77]:
data['tweet'] = data['tweet'].progress_apply(lambda x:split_words(x))

100%|███████████████████████████████| 368413/368413 [00:03<00:00, 110322.54it/s]


In [78]:
print(count_link(data), count_i(data), count_my(data))

195868 218561 87929


### Remove words with single letter

In [79]:
#find number of words with only one letter
tweets = np.array(data['tweet'])
count = 0
word_list = []
for tweet in tweets:
    words = tweet.split()
    for word in words:
        if len(word) <= 1:
            word_list.append(word)
            count += 1
    
    
count

277156

In [80]:
def remove_single_words(text):
    single_words = ['o', 'd', 'r', 'k', 'e', 'y', 'n', 'b', 'c', 'h', 'a', 's', 'g', 'u', 'p', 'w', 'z', 'm', 'x', 'f',
                'j', 'v', 'l', 't', 'q']
    words = text.split()
    lst = []
    for word in words:
        if word not in single_words:
            lst.append(word)
    return ' '.join(lst)
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x : remove_single_words(x))
print(data['tweet'][5003])
print(data['tweet'][5006]) 

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 192666.18it/s]


my wife stopped grocery store curbside pickup they put your orderer your trunk contact she she saw tons boomers going store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also workers watch hands curbside delivery


In [81]:
print(count_link(data), count_i(data), count_my(data))

195868 218561 87929


### Lemmatization

In [82]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk

In [83]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
tqdm.pandas()
data["tweet"] = data["tweet"].progress_apply(lambda text: lemmatize_words(text))
print(data['tweet'][5003])
print(data['tweet'][5006])

100%|█████████████████████████████████| 368413/368413 [03:10<00:00, 1935.42it/s]

my wife stop grocery store curbside pickup they put your orderer your trunk contact she she saw ton boomer go store protection all
they also say it probably safer get food restaurant curb side versus grocery shopping right now less exposure they close also worker watch hand curbside delivery


In [84]:
print(count_link(data), count_i(data), count_my(data))

195869 218569 87929


### Spelling correction

In [85]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [86]:
#Finding words with wrong spelling

In [87]:
words = ' '.join(data['tweet']).split()
misspelled_words = spell.unknown(words)

In [88]:
misspelled_words = list(set(misspelled_words))
len(misspelled_words)

55552

In [93]:
corrected_words = {}
for i in tqdm(misspelled_words):
    corrected = spell.correction(i)
    if corrected != None and i != corrected:
        corrected_words[i] = corrected

100%|███████████████████████████████████| 55552/55552 [1:04:12<00:00, 14.42it/s]


In [94]:
len(corrected_words)

35574

In [100]:
def correct_spellings(text, corrected):
    words = text.split(' ')
    new_words = []
    for word in words:
        if word in corrected:
            new_words.append(corrected[word])
        else:
            new_words.append(word)
    return ' '.join(new_words)
            
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x:correct_spellings(x, corrected_words))        

100%|███████████████████████████████| 368413/368413 [00:01<00:00, 361480.56it/s]


In [101]:
print(count_link(data), count_i(data), count_my(data))

195869 220145 89060


### Remove non-english words

In [102]:
import nltk

In [103]:
#find non-english words
non_english = []
eng_words = set(nltk.corpus.words.words())
words = ' '.join(data['tweet']).split()
for i in tqdm(words):
    if i.lower() not in eng_words and i.lower() not in non_english:
        non_english.append(i)
len(non_english)        

100%|█████████████████████████████| 6610274/6610274 [00:20<00:00, 322165.51it/s]


34028

In [104]:
import nltk

eng_words = set(nltk.corpus.words.words())
keep = ['timelsots','timeslot','convenience','efficient','dissapoint','convenient','inconvenience','substitute',
       'recycling', 'recycle', 'pandemic','omnichannel', 'covid', 'curbside','hyperlink','biodegradable', 'hyperlink']

def remove_nonenglish_words(sent,eng_words,keep):
    words = [w for w in nltk.wordpunct_tokenize(sent) if w.lower() in eng_words or w.lower() in keep]
    return ' '.join(words)

In [105]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x:remove_nonenglish_words(x,eng_words,keep))
print(data['tweet'][5003])
print(data['tweet'][5006])

100%|███████████████████████████████| 368413/368413 [00:02<00:00, 155129.88it/s]

my wife stop grocery store curbside pickup they put your ordered your trunk contact she she saw ton boomer go store protection all
they also say it probably get food restaurant curb side versus grocery shopping right now less exposure they close also worker watch hand curbside delivery


### Find words related to Covid

In [106]:
def replace_curb(text):
    text = text.replace('curb side', 'curbside')
    return text

In [107]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x: replace_curb(x))

100%|██████████████████████████████| 368413/368413 [00:00<00:00, 1488164.99it/s]


In [108]:
def replace_covid(text):
    text = text.replace('corona', 'covid')
    return text

In [109]:
tqdm.pandas()
data['tweet'] = data['tweet'].progress_apply(lambda x: replace_covid(x))

100%|██████████████████████████████| 368413/368413 [00:00<00:00, 1496361.95it/s]


In [110]:
words = ' '.join(data['tweet']).split()

In [111]:
len(data)

368413

In [112]:
data['Category'].value_counts()

Consumer         8377
Advertisement    8092
Name: Category, dtype: int64

In [113]:
data.to_csv('Cleaned_data_for_classification.csv', index = False)